In [74]:
import tensorflow as tf

import numpy as np
from tensorflow.keras import layers
import tensorflow.keras.backend as K

import ssl
from tensorflow.keras import datasets, layers, models

In [75]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [76]:
NUM_EPOCH = 200

# training SNR
eb_n0 = 10
channel_snr = 10
rate = 1 # code rate 
channel_type = 'fading'


#trn, tst = utils.get_mnist()

# ##### KSG MI estimator #######################
# settings = {'kraskov_k': 3}
# gpu_est = est.OpenCLKraskovMI(settings = settings)
# # usage: MI = estimator.estimate(var1, var2)

##### setting for communication channel ######

def getnoisevariance(SNR,rate,P=1):
    # the SNR in args[0] is actually EbN0
    snrdB = SNR + 10*np.log10(rate)
    snr = 10.0**(snrdB/10.0)
    #P_avg = 1
    N0 = P/snr
    return (N0/2)

noise_var = getnoisevariance(eb_n0,rate,P=1)

##############################################


##############################################

label_permutation = np.arange(10)
LABEL_FIRST_HALF = label_permutation[:5]
label2_list = np.zeros((6,5))
for i in range(6):
    label2_list[i][:] = label_permutation[i:i+5]
#LABEL_SECOND_HALF = [5,6,7,8,9]

In [77]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# normalization to 1
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255
# x_train = (x_train / 255.0)
# x_test = (x_test / 255.0)
# reshape to 1d
# x_train = x_train.reshape(x_train.shape[0], -1)
# x_test = x_test.reshape(x_test.shape[0], -1)

# save: original classification
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)


In [18]:
train_filter = np.where(np.in1d(y_train, LABEL_FIRST_HALF))
test_filter =  np.where(np.in1d(y_test, LABEL_FIRST_HALF))

x_1train = x_train[train_filter]
Y_1train = Y_train[train_filter]
x_1test = x_test[test_filter]
Y_1test = Y_test[test_filter]

In [19]:
### normalized Input before 
# class GaussianNoiseLayer(layers.Layer):
#     def __init__(self, stddev, **kwargs):
#         super(GaussianNoiseLayer, self).__init__(**kwargs)
#         self.stddev = stddev
    
#     def call(self, inputs, training=None):
#         if training:
#             noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
#             return inputs + noise
#         else:
#             noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
#             return inputs + noise
    
#     def get_config(self):
#         config = super(GaussianNoiseLayer, self).get_config()
#         config.update({'noise_var': self.stddev})
#         return config

In [20]:
## fading channel

def fading(x, stddev, h=None):
    """Implements the fading channel with multiplicative fading and
    additive white gaussian noise.
    Args:
        x: channel input symbols
        stddev: standard deviation of noise
    Returns:
        y: noisy channel output symbols
    """
    # channel gain
    if h is None:
        h = tf.complex(
            tf.random.normal([tf.shape(x)[0], 1], 0, 1 / np.sqrt(2)),
            tf.random.normal([tf.shape(x)[0], 1], 0, 1 / np.sqrt(2)),
        )

    # additive white gaussian noise
    awgn = tf.complex(
        tf.random.normal(tf.shape(x), 0, 1 / np.sqrt(2)),
        tf.random.normal(tf.shape(x), 0, 1 / np.sqrt(2)),
    )

    return (h * x + stddev * awgn), h

class Channel(layers.Layer):

#cite D-JSCC

    def __init__(self, channel_type, channel_snr, name="channel", **kwargs):
        super(Channel, self).__init__(name=name, **kwargs)
        self.channel_type = channel_type
        self.channel_snr = channel_snr

    def call(self, inputs):
        (encoded_img, prev_h) = inputs
        inter_shape = tf.shape(encoded_img)
        # reshape array to [-1, dim_z]
        z = layers.Flatten()(encoded_img)
        # convert from snr to std
        # print("channel_snr: {}".format(self.channel_snr))
        noise_stddev = np.sqrt(10 ** (-self.channel_snr / 10))

        # Add channel noise
        if self.channel_type == "awgn":
            dim_z = tf.shape(z)[1]
            # normalize latent vector so that the average power is 1
            z_in = tf.sqrt(tf.cast(dim_z, dtype=tf.float32)) * tf.nn.l2_normalize(
                z, axis=1
            )
            z_out = real_awgn(z_in, noise_stddev)
            h = tf.ones_like(z_in)  # h just makes sense on fading channels

        elif self.channel_type == "fading":
            dim_z = tf.shape(z)[1] // 2
            # convert z to complex representation
            z_in = tf.complex(z[:, :dim_z], z[:, dim_z:])
            # normalize the latent vector so that the average power is 1
            z_norm = tf.reduce_sum(
                tf.math.real(z_in * tf.math.conj(z_in)), axis=1, keepdims=True
            )
            z_in = z_in * tf.complex(
                tf.sqrt(tf.cast(dim_z, dtype=tf.float32) / z_norm), 0.0
            )
            z_out, h = fading(z_in, noise_stddev, prev_h)
            # convert back to real
            z_out = tf.concat([tf.math.real(z_out), tf.math.imag(z_out)], 1)

        # convert signal back to intermediate shape
        z_out = tf.reshape(z_out, inter_shape)

        return z_out, h


In [21]:
class CustomMetricCallback(tf.keras.callbacks.Callback):
    def __init__(self, display_frequency=10):
        super(CustomMetricCallback, self).__init__()
        self.display_frequency = display_frequency

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.display_frequency == 0:
            metrics_str = " - ".join([f"{metric_name}: {value:.4f}" for metric_name, value in logs.items()])
            print(f"Epoch {epoch}/{self.params['epochs']} - {metrics_str}")

            
callback = CustomMetricCallback(display_frequency=50)

In [26]:

def do_report(epoch):
#     if epoch < 20:       # Log for all first 20 epochs
#         return True
    if epoch < 100:    # Then for every 5th epoch
        return (epoch % 5 == 0)
    elif epoch < 200:    # Then every 10th
        return (epoch % 10 == 0)
    else:                # Then every 100th
        return (epoch % 100 == 0)
    
    
def train_first_stage(lambda_val):

    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    prev_chn_gain = None

        ############ Model Structure ###############################

    input_layer  = tf.keras.layers.Input(shape=x_train[0].shape)
    encoder_1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(input_layer)
    maxpool_1 = layers.MaxPooling2D((2, 2))(encoder_1)
    encoder_2 = layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)(encoder_1)
    maxpool_2 = layers.MaxPooling2D((2, 2))(encoder_2)

    normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2)
    fadingchannel = Channel(channel_type, channel_snr, name="channel_output")
    noise_layer, chn_gain = fadingchannel((normalized_x, prev_chn_gain))
    #noise_layer = GaussianNoiseLayer(stddev = sd)(normalized_x)

    ## first loss: classification 
#     CE_decoder_1 =  layers.Conv2D(32, (3, 3), activation='relu')(noise_layer)
#     CE_maxpool_1 = layers.MaxPooling2D((2, 2))(CE_decoder_1)
    # CE_decoder_2 =  layers.Conv2D(32, (3, 3), activation='relu')(CE_maxpool_1)
    # CE_maxpool_2 = layers.MaxPooling2D((2, 2))(CE_decoder_2)
    flatten_1 = layers.Flatten()(noise_layer)
    CE_dense_2 = layers.Dense(40, activation='relu')(flatten_1)
    CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_2)
    #CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

    ## second loss: reconstruction
    mse_1 = layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same')(noise_layer)
    mse_2 = layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same')(mse_1)
    mse_output = layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same',name='mse')(mse_2)

    #    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    #    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1)  

    model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(optimizer=opt,
                  loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'},
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()},
                  loss_weights=[1, lambda_val])


    SNR = eb_n0
    #     print("current snr: ",SNR)
    # #     print("====================")
    #     reporter = getMIOutput(trn=x_1train, 
    #                                tst=x_1test, 
    #                                snr=SNR,
    #                               do_save_func=do_report)

    history = model.fit(x=x_1train, y=(Y_1train,x_1train),
                        batch_size=128,
                        epochs=100,
                        verbose=0,
                        validation_data=(x_1test, (Y_1test,x_1test)))
                       #callbacks = [callback])
                        #callbacks=[reporter,])
    print("acc:", history.history['val_CE_accuracy'][-1]) 


#     ep_last = len(reporter.datalist) -1
#     # Z_data = reporter.datalist[EPOCH]['data']['activity_tst'][3][:10000]
#     ZHAT_data = reporter.datalist[ep_last]['data']['activity_tst'][4][:SAMPLE_NUM]
#     x_selected = funcs.normalize_data_new(x_1train[:SAMPLE_NUM],C=10)[0]
#     print("Current I(Z_hat; X) is: ",gpu_est.estimate(funcs.normalize_data_new(ZHAT_data,C=4)[0],x_selected))

  
    ################### secpmd stage #################
    print("stage 2!")
   
    train_filter = np.where(np.in1d(y_train, LABEL_SECOND_HALF))
    test_filter =  np.where(np.in1d(y_test, LABEL_SECOND_HALF))

    x_retrain = x_train[train_filter]
    x_retest = x_test[test_filter]
    Y_retrain = Y_train[train_filter]
    Y_retest = Y_test[test_filter]

    flatten_2 = layers.Flatten()(noise_layer)
    CE_dense_3 = layers.Dense(40, activation='relu')(flatten_2)
    CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_2)
    #     CE_decoder_3 = tf.keras.layers.Dense(64, activation='relu')(noise_layer)
    #     CE_decoder_4 = tf.keras.layers.Dense(64, activation='relu')(CE_decoder_3)
    #     CE_output1 = tf.keras.layers.Dense(10, activation='softmax', name='CE')(CE_decoder_4)

    reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])

    reconstructed_model.layers[1].trainable = False
    reconstructed_model.layers[2].trainable = False

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)


    reconstructed_model.compile(optimizer=opt, 
                loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'
                         },              loss_weights=[1, 0],
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()
                           }
                 )

    #     reporter_2 = getMIOutput(trn=x_retrain, 
    #                                tst=Y_retrain, 
    #                                snr=SNR,
    #                               do_save_func=do_report)

    history1 = reconstructed_model.fit(x=x_retrain, y=(Y_retrain,x_retrain),
                        batch_size=256,
                        epochs=100,
                        verbose=0,
                        validation_data=(x_retest, (Y_retest,x_retest)))
                        # callbacks=[callback])
    print("acc:", history1.history['val_CE_accuracy'][-1]) 
    return history.history['val_CE_accuracy'][-1], history1.history['val_CE_accuracy'][-1]
    

In [27]:
sd = np.sqrt(noise_var)

lambda_list = [0,1,3,10]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2024-02-22 10:34:31.363129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2024-02-22 10:34:31.621891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-22 10:34:31.888633: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f9474312ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-22 10:34:31.888678: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-02-22 10:34:32.024950: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


acc: 0.9861840605735779
stage 2!
acc: 0.9642049074172974
lambda:  1
acc: 0.9842382073402405
stage 2!
acc: 0.9642049074172974
lambda:  3
acc: 0.9842382073402405
stage 2!
acc: 0.9605019688606262
lambda:  10
acc: 0.9828760623931885
stage 2!
acc: 0.9602962136268616
current overlap is:  1
lambda:  0
acc: 0.9861840605735779
stage 2!
acc: 0.974555253982544
lambda:  1
acc: 0.9877408146858215
stage 2!
acc: 0.9760032892227173
lambda:  3
acc: 0.984627366065979
stage 2!
acc: 0.970417857170105
lambda:  10
acc: 0.9782058596611023
stage 2!
acc: 0.9635912179946899
current overlap is:  2
lambda:  0
acc: 0.9875462055206299
stage 2!
acc: 0.9743326306343079
lambda:  1
acc: 0.9879354238510132
stage 2!
acc: 0.9755646586418152
lambda:  3
acc: 0.9789842367172241
stage 2!
acc: 0.9720739126205444
lambda:  10
acc: 0.9813193082809448
stage 2!
acc: 0.9708418846130371
current overlap is:  3
lambda:  0
acc: 0.9838489890098572
stage 2!
acc: 0.9772260785102844
lambda:  1
acc: 0.9867678284645081
stage 2!
acc: 0.9759950

In [69]:
#print(np.reshape(acc2_list,(6,-1)))
print(np.mean(np.reshape(acc2_list,(6,-1)),axis=0))

[0.97576074 0.9758976  0.97327621 0.96985759]


In [54]:
ave = []
for i in range(1000):
    x= np.random.normal(0,0.5) 
    y= np.random.normal(0,0.5) 
    #print(x)

    ave.append(np.sqrt(x**2 + y**2))

In [73]:
np.mean(1/np.array(ave))

2.451984914322634

In [64]:
np.sum(np.array(ave)>1)

159

In [22]:
np.reshape(acc2_list,(6,-1))

array([[0.83920002, 0.85339999, 0.84380001, 0.83960003, 0.85000002,
        0.8484    , 0.86400002, 0.85839999],
       [0.79500002, 0.79939997, 0.80779999, 0.78799999, 0.80900002,
        0.80800003, 0.7974    , 0.80400002],
       [0.68919998, 0.66939998, 0.6796    , 0.69300002, 0.67720002,
        0.68300003, 0.68220001, 0.69      ],
       [0.63480002, 0.63239998, 0.6318    , 0.63620001, 0.6462    ,
        0.62639999, 0.6462    , 0.653     ],
       [0.70060003, 0.69440001, 0.68519998, 0.70880002, 0.70060003,
        0.6904    , 0.6918    , 0.69980001],
       [0.76160002, 0.76959997, 0.76959997, 0.76160002, 0.77640003,
        0.76840001, 0.76440001, 0.77020001]])

In [17]:
### please given data ###

lines = data.strip().split('\n')
array1 = []
array2 = []
is_stage2_next = False

for i in range(len(lines)):
    if "stage 2!" in lines[i]:
        is_stage2_next = True
        continue
    
    if lines[i].startswith("acc:"):
        acc_value = float(lines[i].split(":")[1])
        if is_stage2_next:
            array2.append(acc_value)
            is_stage2_next = False
        else:
            array1.append(acc_value)

print("Array 1:", array1)
print("Array 2:", array2)







Array 1: [0.7185999751091003, 0.7432000041007996, 0.7235999703407288, 0.7214000225067139, 0.728600025177002, 0.7211999893188477, 0.7229999899864197, 0.7287999987602234, 0.7129999995231628, 0.7084000110626221, 0.7128000259399414, 0.7134000062942505, 0.7185999751091003, 0.7369999885559082, 0.7264000177383423, 0.7347999811172485, 0.7164000272750854, 0.7103999853134155, 0.723800003528595, 0.7139999866485596, 0.7251999974250793, 0.7251999974250793, 0.7253999710083008, 0.7287999987602234, 0.7006000280380249, 0.7088000178337097, 0.715399980545044, 0.725600004196167, 0.7282000184059143, 0.7269999980926514, 0.7229999899864197, 0.7192000150680542, 0.7232000231742859, 0.7113999724388123, 0.7235999703407288, 0.728600025177002, 0.7289999723434448, 0.7354000210762024, 0.730400025844574, 0.7319999933242798, 0.7206000089645386, 0.7211999893188477, 0.7218000292778015, 0.717199981212616, 0.715399980545044, 0.7325999736785889, 0.7283999919891357, 0.7210000157356262]
Array 2: [0.8004000186920166, 0.822000

In [6]:
np.reshape(array1,(6,-1))
#print(np.reshape(array2,(6,-1)))

In [9]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2023-08-11 13:31:20.039462: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78836 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-08-11 13:31:23.317045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-11 13:31:23.562262: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-11 13:31:23.656432: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7faf088588c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-11 13:31:23.656474: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-08-11 13:31:23.735796: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled clus

acc: 0.7039999961853027
stage 2!
acc: 0.7901999950408936
lambda:  0.2
acc: 0.7228000164031982
stage 2!
acc: 0.8159999847412109
lambda:  0.5
acc: 0.7152000069618225
stage 2!
acc: 0.8037999868392944
lambda:  1
acc: 0.7235999703407288
stage 2!
acc: 0.8091999888420105
lambda:  3
acc: 0.7178000211715698
stage 2!
acc: 0.8068000078201294
lambda:  5
acc: 0.7300000190734863
stage 2!
acc: 0.8203999996185303
lambda:  10
acc: 0.7332000136375427
stage 2!
acc: 0.8252000212669373
lambda:  15
acc: 0.7215999960899353
stage 2!
acc: 0.8154000043869019
current overlap is:  1
lambda:  0
acc: 0.7175999879837036
stage 2!
acc: 0.7746000289916992
lambda:  0.2
acc: 0.7203999757766724
stage 2!
acc: 0.7853999733924866
lambda:  0.5
acc: 0.728600025177002
stage 2!
acc: 0.7770000100135803
lambda:  1
acc: 0.7184000015258789
stage 2!
acc: 0.7752000093460083
lambda:  3
acc: 0.7283999919891357
stage 2!
acc: 0.775600016117096
lambda:  5
acc: 0.7342000007629395
stage 2!
acc: 0.7832000255584717
lambda:  10
acc: 0.733200013

In [9]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2023-08-12 04:55:05.857821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1131 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-08-12 04:55:08.293360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-12 04:55:08.483354: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 891.11MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-12 04:55:08.486133: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 891.11MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-12 0

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv2d_1/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    F

In [10]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2023-08-12 08:24:50.058525: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 292.97MiB (rounded to 307200000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-12 08:24:50.058600: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-12 08:24:50.058610: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 68, Chunks in use: 68. 17.0KiB allocated for chunks. 17.0KiB in use in bin. 1.9KiB client-requested in use in bin.
2023-08-12 08:24:50.058620: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 3, Chunks in use: 3. 1.5KiB allocated for chunks. 1.5KiB in use in bin. 1.1KiB client-requested in use in bin.
2023-08-12 08:24:50.058626: I tensorflow/tsl/framework

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

low/tsl/framework/bfc_allocator.cc:1103] 3 Chunks of size 512 totalling 1.5KiB
2023-08-12 08:24:50.059319: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 4 Chunks of size 1280 totalling 5.0KiB
2023-08-12 08:24:50.059324: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 2 Chunks of size 2048 totalling 4.0KiB
2023-08-12 08:24:50.059330: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 2560 totalling 2.5KiB
2023-08-12 08:24:50.059335: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 3 Chunks of size 10496 totalling 30.8KiB
2023-08-12 08:24:50.059340: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 20480 totalling 20.0KiB
2023-08-12 08:24:50.059346: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 26368 totalling 25.8KiB
2023-08-12 08:24:50.059351: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 32768 totalling 32.0KiB
2023-08-12 08:24:50.059356: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 8 Chunks of size

In [10]:
acc1_list

NameError: name 'acc1_list' is not defined

In [9]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
for i in range(6):
    print("current overlap is: ", (5) )
    LABEL_SECOND_HALF = label2_list[5]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        train_first_stage(lambda_val)
        #train_second_stage()

current overlap is:  5
lambda:  0


2023-08-12 08:26:59.207227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78836 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-08-12 08:27:02.458515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-12 08:27:02.697554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-12 08:27:02.786255: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55ef60963f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-12 08:27:02.786299: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-08-12 08:27:02.850942: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled clus

acc: 0.7545999884605408
stage 2!
acc: 0.7680000066757202
lambda:  0.2
acc: 0.7680000066757202
stage 2!
acc: 0.7738000154495239
lambda:  0.5
acc: 0.7605999708175659
stage 2!
acc: 0.7688000202178955
lambda:  1
acc: 0.748199999332428
stage 2!
acc: 0.7626000046730042
lambda:  3
acc: 0.7738000154495239
stage 2!
acc: 0.7835999727249146
lambda:  5
acc: 0.7680000066757202
stage 2!
acc: 0.777999997138977
lambda:  10
acc: 0.7703999876976013
stage 2!
acc: 0.7788000106811523
lambda:  15
acc: 0.7767999768257141
stage 2!
acc: 0.7857999801635742
current overlap is:  5
lambda:  0
acc: 0.7581999897956848
stage 2!
acc: 0.7694000005722046
lambda:  0.2
acc: 0.7487999796867371
stage 2!
acc: 0.7656000256538391
lambda:  0.5
acc: 0.7563999891281128
stage 2!
acc: 0.7645999789237976
lambda:  1
acc: 0.7572000026702881
stage 2!
acc: 0.7742000222206116
lambda:  3
acc: 0.7752000093460083
stage 2!
acc: 0.7918000221252441
lambda:  5
acc: 0.769599974155426
stage 2!
acc: 0.7666000127792358
lambda:  10
acc: 0.7688000202